In [19]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import random
import os

In [20]:
df = pd.read_csv('train_set.csv')
df

,id_img,path,label
0,0,../data/images/train/angry\0.jpg,angry
1,1,../data/images/train/angry\1.jpg,angry
2,10,../data/images/train/angry\10.jpg,angry
3,10002,../data/images/train/angry\10002.jpg,angry
4,10016,../data/images/train/angry\10016.jpg,angry
...,...,...,...
28816,9969,../data/images/train/surprise\9969.jpg,surprise
28817,9985,../data/images/train/surprise\9985.jpg,surprise
28818,9990,../data/images/train/surprise\9990.jpg,surprise
28819,9992,../data/images/train/surprise\9992.jpg,surprise


In [21]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df['target'] = label_encoder.fit_transform(df['label'])

In [22]:
df['target'].value_counts()

target
3    7164
4    4982
5    4938
2    4103
0    3993
6    3205
1     436
Name: count, dtype: int64

In [23]:
df['path'] = df['path'].str.replace('\\', '/')
df['path'] = df['path'].str.replace('../data/images/train/', './data/images/train/')

def cargar_y_convertir_imagen(ruta_imagen):
    image = cv2.imread(ruta_imagen, 0)
    smallimage = cv2.resize(image, (32, 32))
    image_array = np.array(smallimage)
    return image_array

df['imagenes'] =df['path'].apply(cargar_y_convertir_imagen)

In [8]:
df['path'] = df['path'].str.replace('\\', '/')
df['path'] = df['path'].str.replace('../data/images/train/', './data/images/train/')
def cargar_y_convertir_imagen(ruta_imagen):
    image = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)  
    smallimage = cv2.resize(image, (32, 32))
    
    image_rgb = cv2.cvtColor(smallimage, cv2.COLOR_GRAY2RGB)
    
    return image_rgb
df['imagenes'] = df['path'].apply(cargar_y_convertir_imagen)

In [24]:
from sklearn.utils import shuffle
df = shuffle(df, random_state=42)

In [32]:
df['imagenes'][1].shape

(32, 32, 1)

In [29]:
df['imagenes'] = df['imagenes'].apply(lambda x: np.array(x).reshape(32, 32,1))

X = np.stack(df['imagenes'].to_numpy())
y = df['target']

X = X / 255.0 

In [30]:
from keras.callbacks import ModelCheckpoint
mcheckpoint = ModelCheckpoint("callback_model.h5")

In [31]:
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(patience=5)

In [16]:
modelo_cnn = Sequential()
modelo_cnn.add(Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 1)))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Conv2D(128, (3, 3), activation='relu'))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Flatten())
modelo_cnn.add(Dense(128, activation='relu'))
modelo_cnn.add(Dropout(0.5))
modelo_cnn.add(Dense(64, activation='relu'))
modelo_cnn.add(Dropout(0.5))
modelo_cnn.add(Dense(7, activation='sigmoid'))  

opt = Adam(learning_rate=0.0001)
modelo_cnn.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

modelo_cnn.summary()

modelo_cnn.fit(X, y, epochs=50, batch_size=48, validation_split=0.2, callbacks=[earlystop, mcheckpoint])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 30, 30, 64)        1792      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 15, 15, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 128)       73856     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 6, 6, 128)         0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 4608)              0         
                                                                 
 dense_3 (Dense)             (None, 128)              

c:\Users\migue\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


481/481 [==============================] - 12s 24ms/step - loss: 1.8347 - accuracy: 0.2365 - val_loss: 1.7861 - val_accuracy: 0.2487
Epoch 3/50
481/481 [==============================] - 11s 24ms/step - loss: 1.7740 - accuracy: 0.2829 - val_loss: 1.6987 - val_accuracy: 0.3627
Epoch 4/50
481/481 [==============================] - 11s 24ms/step - loss: 1.7167 - accuracy: 0.3204 - val_loss: 1.6482 - val_accuracy: 0.3979
Epoch 5/50
481/481 [==============================] - 12s 24ms/step - loss: 1.6635 - accuracy: 0.3464 - val_loss: 1.5733 - val_accuracy: 0.3913
Epoch 6/50
481/481 [==============================] - 12s 25ms/step - loss: 1.6251 - accuracy: 0.3673 - val_loss: 1.5397 - val_accuracy: 0.4097
Epoch 7/50
481/481 [==============================] - 12s 25ms/step - loss: 1.5947 - accuracy: 0.3778 - val_loss: 1.5057 - val_accuracy: 0.4394
Epoch 8/50
481/481 [==============================] - 12s 24ms/step - loss: 1.5636 - accuracy: 0.3888 - val_loss: 1.4765 - val_accuracy: 0.4411
Epo

In [33]:
#prueba con 48x48x3
modelo_cnn = Sequential()
modelo_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Conv2D(64, (3, 3), activation='relu'))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Conv2D(128, (3, 3), activation='relu'))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Flatten())
modelo_cnn.add(Dense(256, activation='relu'))
modelo_cnn.add(Dropout(0.5))
modelo_cnn.add(Dense(128, activation='relu'))
modelo_cnn.add(Dropout(0.5))
modelo_cnn.add(Dense(7, activation='sigmoid'))  

opt = Adam(learning_rate=0.0001)
modelo_cnn.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

modelo_cnn.summary()

modelo_cnn.fit(X, y, epochs=50, batch_size=32, validation_split=0.2, callbacks=[earlystop, mcheckpoint])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 30, 30, 32)        320       
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 15, 15, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 6, 6, 64)          0         
 ng2D)                                                           
                                                                 
 conv2d_12 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 2, 2, 128)        

c:\Users\migue\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


721/721 [==============================] - 7s 9ms/step - loss: 1.8081 - accuracy: 0.2477 - val_loss: 1.7712 - val_accuracy: 0.2597
Epoch 3/50
721/721 [==============================] - 7s 9ms/step - loss: 1.7463 - accuracy: 0.2887 - val_loss: 1.6579 - val_accuracy: 0.3481
Epoch 4/50
721/721 [==============================] - 6s 9ms/step - loss: 1.6706 - accuracy: 0.3394 - val_loss: 1.6008 - val_accuracy: 0.3821
Epoch 5/50
721/721 [==============================] - 6s 9ms/step - loss: 1.6234 - accuracy: 0.3604 - val_loss: 1.5653 - val_accuracy: 0.4071
Epoch 6/50
721/721 [==============================] - 6s 9ms/step - loss: 1.5845 - accuracy: 0.3844 - val_loss: 1.5432 - val_accuracy: 0.4023
Epoch 7/50
721/721 [==============================] - 6s 9ms/step - loss: 1.5517 - accuracy: 0.4004 - val_loss: 1.4992 - val_accuracy: 0.4206
Epoch 8/50
721/721 [==============================] - 6s 9ms/step - loss: 1.5235 - accuracy: 0.4123 - val_loss: 1.4769 - val_accuracy: 0.4390
Epoch 9/50
721/72